In [1]:
print("hello world")

hello world


# Flask Error Handling and Debugging Notebook

In [3]:
# Run this cell first to install required packages (if needed)
# !pip install flask flask-debugtoolbar

1. Introduction to Flask Error Handling

This notebook covers comprehensive error handling and debugging techniques for Flask applications.


2. Common Types of Errors in Flask


2.1 HTTP Status Code Errors

In [4]:
from IPython.display import Markdown

http_errors = """
| Code | Name | Description |
|------|------|-------------|
| 400 | Bad Request | Invalid client request |
| 401 | Unauthorized | Authentication required |
| 403 | Forbidden | Server refuses action |
| 404 | Not Found | Resource doesn't exist |
| 405 | Method Not Allowed | Wrong HTTP method |
| 500 | Internal Server Error | Server-side error |
| 502 | Bad Gateway | Invalid response from upstream |
| 503 | Service Unavailable | Server overloaded/maintenance |
"""

display(Markdown("### Common HTTP Errors"))
display(Markdown(http_errors))

### Common HTTP Errors


| Code | Name | Description |
|------|------|-------------|
| 400 | Bad Request | Invalid client request |
| 401 | Unauthorized | Authentication required |
| 403 | Forbidden | Server refuses action |
| 404 | Not Found | Resource doesn't exist |
| 405 | Method Not Allowed | Wrong HTTP method |
| 500 | Internal Server Error | Server-side error |
| 502 | Bad Gateway | Invalid response from upstream |
| 503 | Service Unavailable | Server overloaded/maintenance |


## 2.2 Python Runtime Errors

In [5]:
python_errors = """
| Exception | Description |
|-----------|-------------|
| TypeError | Wrong data type operation |
| ValueError | Invalid value |
| KeyError | Missing dictionary key |
| AttributeError | Missing object attribute |
| IndexError | List index out of range |
| NameError | Undefined variable |
| ImportError | Failed module import |
| ZeroDivisionError | Division by zero |
"""

display(Markdown("### Common Python Exceptions"))
display(Markdown(python_errors))

### Common Python Exceptions


| Exception | Description |
|-----------|-------------|
| TypeError | Wrong data type operation |
| ValueError | Invalid value |
| KeyError | Missing dictionary key |
| AttributeError | Missing object attribute |
| IndexError | List index out of range |
| NameError | Undefined variable |
| ImportError | Failed module import |
| ZeroDivisionError | Division by zero |


### Flask-Specific Errors

# Flask Common Issues

404 - Route not found

405 - Method not allowed for route

400 - Bad request (form validation)

500 - Template rendering errors

500 - Database connection issues

### 3. Flask Application Setup with Debugging

### 3.1 Basic Flask App with Debug Configuration

In [6]:
from flask import Flask, render_template, request, jsonify
import logging
from werkzeug.exceptions import HTTPException
import os

# Create Flask application
app = Flask(__name__)

# Configuration
class Config:
    SECRET_KEY = 'dev-key-123'
    DEBUG = True
    TESTING = True

app.config.from_object(Config)

# Setup logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print("Flask app configured with debug mode enabled")

Flask app configured with debug mode enabled


### 3.2 Development vs Production Configuration

In [7]:
display(Markdown("### Configuration Differences"))

config_table = """
| Setting | Development | Production |
|---------|-------------|------------|
| DEBUG | True | False |
| TESTING | True | False |
| ENV | 'development' | 'production' |
| Secret Key | Simple string | Environment variable |
| Error Display | Detailed debugger | User-friendly pages |
"""

display(Markdown(config_table))

### Configuration Differences


| Setting | Development | Production |
|---------|-------------|------------|
| DEBUG | True | False |
| TESTING | True | False |
| ENV | 'development' | 'production' |
| Secret Key | Simple string | Environment variable |
| Error Display | Detailed debugger | User-friendly pages |


### 4. Custom Error Handlers
### 4.1 Basic Error Handler Implementation

In [8]:
# Custom exception class
class CustomError(Exception):
    def __init__(self, message, status_code=400):
        super().__init__()
        self.message = message
        self.status_code = status_code

# Error handlers
@app.errorhandler(404)
def not_found_error(error):
    if request.accept_mimetypes.accept_json and not request.accept_mimetypes.accept_html:
        return jsonify({'error': 'Resource not found'}), 404
    return "<h1>404 - Page Not Found</h1><p>The page you're looking for doesn't exist.</p>", 404

@app.errorhandler(500)
def internal_error(error):
    app.logger.error(f'500 Error: {error}')
    if request.accept_mimetypes.accept_json and not request.accept_mimetypes.accept_html:
        return jsonify({'error': 'Internal server error'}), 500
    return "<h1>500 - Internal Server Error</h1><p>Something went wrong on our end.</p>", 500

@app.errorhandler(CustomError)
def handle_custom_error(error):
    response = jsonify({
        'error': error.message,
        'status_code': error.status_code
    })
    response.status_code = error.status_code
    return response

@app.errorhandler(Exception)
def handle_generic_error(error):
    app.logger.error(f'Unhandled exception: {error}')
    if isinstance(error, HTTPException):
        return error
    
    return jsonify({
        'error': 'An unexpected error occurred',
        'message': str(error)
    }), 500

print("Custom error handlers registered")

Custom error handlers registered


### 4.2 Testing Error Handlers

In [9]:
# Routes to test error handling
@app.route('/')
def index():
    return "<h1>Welcome to Flask Error Handling Demo</h1><ul>" \
           "<li><a href='/divide/10/2'>Working division</a></li>" \
           "<li><a href='/divide/10/0'>Division by zero</a></li>" \
           "<li><a href='/nonexistent'>404 page</a></li>" \
           "<li><a href='/server-error'>500 error</a></li>" \
           "</ul>"

@app.route('/divide/<int:a>/<int:b>')
def divide_numbers(a, b):
    """Example route with error handling"""
    try:
        if b == 0:
            raise CustomError("Division by zero is not allowed", 400)
        
        result = a / b
        return jsonify({
            'operation': f'{a} / {b}',
            'result': result,
            'status': 'success'
        })
    
    except CustomError:
        raise  # Re-raise custom errors
    except Exception as e:
        app.logger.error(f"Unexpected error in division: {e}")
        raise CustomError("An error occurred during division", 500)

@app.route('/server-error')
def cause_server_error():
    """Route that intentionally causes a server error"""
    # This will raise a NameError
    return undefined_variable  # This variable doesn't exist

@app.route('/user/<user_id>')
def get_user(user_id):
    """Route with custom error for missing users"""
    users = {
        '1': {'name': 'John Doe', 'email': 'john@example.com'},
        '2': {'name': 'Jane Smith', 'email': 'jane@example.com'}
    }
    
    if user_id not in users:
        raise CustomError("User not found", 404)
    
    return jsonify(users[user_id])

print("Test routes defined")

Test routes defined


### 5. Debugging Techniques


### 5.1 Using Flask Debug Toolbar

In [10]:
# Uncomment to use Flask Debug Toolbar
"""
from flask_debugtoolbar import DebugToolbarExtension

app.config['DEBUG_TB_ENABLED'] = True
app.config['DEBUG_TB_INTERCEPT_REDIRECTS'] = False
toolbar = DebugToolbarExtension(app)
print("Flask Debug Toolbar enabled")
"""

'\nfrom flask_debugtoolbar import DebugToolbarExtension\n\napp.config[\'DEBUG_TB_ENABLED\'] = True\napp.config[\'DEBUG_TB_INTERCEPT_REDIRECTS\'] = False\ntoolbar = DebugToolbarExtension(app)\nprint("Flask Debug Toolbar enabled")\n'

### 5.2 Strategic Logging Setup

In [11]:
def setup_advanced_logging():
    """Setup comprehensive logging configuration"""
    
    # Create logger
    logger = logging.getLogger('flask_app')
    logger.setLevel(logging.DEBUG)
    
    # Create formatters
    detailed_formatter = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s - %(filename)s:%(lineno)d - %(message)s'
    )
    
    simple_formatter = logging.Formatter(
        '%(levelname)s - %(message)s'
    )
    
    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_handler.setFormatter(simple_formatter)
    
    # File handler
    file_handler = logging.FileHandler('flask_errors.log')
    file_handler.setLevel(logging.DEBUG)
    file_handler.setFormatter(detailed_formatter)
    
    # Add handlers
    logger.addHandler(console_handler)
    logger.addHandler(file_handler)
    
    return logger

# Initialize advanced logging
app_logger = setup_advanced_logging()
app.logger = app_logger

print("Advanced logging setup complete")

Advanced logging setup complete


### 5.3 Debugging with pdb

In [12]:
import pdb

@app.route('/debug-demo')
def debug_demo():
    """Route demonstrating debugging with pdb"""
    app.logger.info("Debug demo route accessed")
    
    data = {
        'items': ['apple', 'banana', 'cherry'],
        'count': 3,
        'status': 'processing'
    }
    
    # Uncomment to activate debugger
    # pdb.set_trace()  # Program will pause here
    
    # Simulate some processing
    try:
        result = process_data(data)
        return jsonify({'result': result, 'status': 'success'})
    except Exception as e:
        app.logger.error(f"Error in debug demo: {e}")
        return jsonify({'error': str(e)}), 500

def process_data(data):
    """Helper function to demonstrate debugging"""
    # This function can be examined in debugger
    total_length = sum(len(item) for item in data['items'])
    processed = {
        'total_chars': total_length,
        'item_count': data['count'],
        'first_item': data['items'][0] if data['items'] else None
    }
    return processed

print("Debugging demo routes defined")

Debugging demo routes defined


### 6. Interactive Testing Section
### 6.1 Test the Application

In [13]:
# This cell creates a test client to demonstrate error handling
with app.test_client() as client:
    display(Markdown("### Testing Error Handling"))
    
    # Test successful request
    response1 = client.get('/divide/10/2')
    print(f"Successful division: Status {response1.status_code}")
    print(f"Response: {response1.get_json()}")
    print()
    
    # Test division by zero
    response2 = client.get('/divide/10/0')
    print(f"Division by zero: Status {response2.status_code}")
    print(f"Response: {response2.get_json()}")
    print()
    
    # Test 404 error
    response3 = client.get('/nonexistent-page')
    print(f"Non-existent page: Status {response3.status_code}")
    print()
    
    # Test user lookup - existing user
    response4 = client.get('/user/1')
    print(f"Existing user: Status {response4.status_code}")
    print(f"Response: {response4.get_json()}")
    print()
    
    # Test user lookup - non-existing user
    response5 = client.get('/user/999')
    print(f"Non-existing user: Status {response5.status_code}")
    print(f"Response: {response5.get_json()}")

### Testing Error Handling

Successful division: Status 200
Response: {'operation': '10 / 2', 'result': 5.0, 'status': 'success'}

Division by zero: Status 400
Response: {'error': 'Division by zero is not allowed', 'status_code': 400}

Non-existent page: Status 404

Existing user: Status 200
Response: {'email': 'john@example.com', 'name': 'John Doe'}

Non-existing user: Status 404
Response: {'error': 'User not found', 'status_code': 404}


### 6.2 Testing Server Errors

In [14]:
# Test server error (this will log an error but won't crash the notebook)
with app.test_client() as client:
    display(Markdown("### Testing Server Errors"))
    
    try:
        response = client.get('/server-error')
        print(f"Server error route: Status {response.status_code}")
        print(f"Response: {response.get_data(as_text=True)}")
    except Exception as e:
        print(f"Exception caught: {e}")

### Testing Server Errors

ERROR - Unhandled exception: name 'undefined_variable' is not defined
2025-10-05 23:39:40,952 - flask_app - ERROR - Unhandled exception: name 'undefined_variable' is not defined


Server error route: Status 500
Response: {
  "error": "An unexpected error occurred",
  "message": "name 'undefined_variable' is not defined"
}



### 7. Advanced Error Handling Patterns
### 7.1 Contextual Error Handling

In [15]:
from contextlib import contextmanager

@contextmanager
def database_operation():
    """Context manager for database operations with error handling"""
    try:
        app.logger.info("Starting database operation")
        yield "mock_database_connection"
        app.logger.info("Database operation completed successfully")
    except Exception as e:
        app.logger.error(f"Database operation failed: {e}")
        raise CustomError("Database operation failed", 500)

@app.route('/api/data')
def get_data_with_context():
    """Route using context manager for error handling"""
    with database_operation() as db:
        # Simulate database operations
        data = simulate_database_query(db)
        return jsonify({'data': data, 'status': 'success'})

def simulate_database_query(db_connection):
    """Simulate database query that might fail"""
    # Simulate random failure for demonstration
    import random
    if random.random() < 0.3:  # 30% chance of failure
        raise ConnectionError("Database connection lost")
    return {'users': [{'id': 1, 'name': 'Test User'}]}

print("Advanced error handling patterns defined")

AssertionError: The setup method 'route' can no longer be called on the application. It has already handled its first request, any changes will not be applied consistently.
Make sure all imports, decorators, functions, etc. needed to set up the application are done before running it.

### 7.2 Error Recovery and Fallbacks

In [ ]:
class FallbackManager:
    """Manager for handling fallback scenarios"""
    
    def __init__(self):
        self.cache = {}
    
    def get_with_fallback(self, key, primary_func, fallback_func):
        """Try primary function, fallback if it fails"""
        try:
            result = primary_func()
            self.cache[key] = result  # Cache successful result
            app.logger.info(f"Primary method successful for {key}")
            return result
        except Exception as e:
            app.logger.warning(f"Primary method failed for {key}, using fallback: {e}")
            # Try to return cached data or use fallback
            if key in self.cache:
                app.logger.info(f"Returning cached data for {key}")
                return self.cache[key]
            return fallback_func()

fallback_manager = FallbackManager()

@app.route('/reliable-data')
def get_reliable_data():
    """Route demonstrating fallback pattern"""
    
    def primary_data_source():
        # Simulate unreliable external service
        import random
        if random.random() < 0.4:  # 40% failure rate
            raise ConnectionError("External service unavailable")
        return {'source': 'primary', 'data': 'fresh_data'}
    
    def fallback_data_source():
        return {'source': 'fallback', 'data': 'cached_data', 'note': 'Service degraded'}
    
    result = fallback_manager.get_with_fallback(
        'reliable_data',
        primary_data_source,
        fallback_data_source
    )
    
    return jsonify(result)

print("Error recovery patterns implemented")

### 8. Running the Application
### 8.1 Start the Flask Development Server

In [16]:
display(Markdown("### Starting Flask Development Server"))

# Note: In Jupyter, we can't run the server in the normal way, but we can demonstrate the code
server_code = """
if __name__ == '__main__':
    print("Starting Flask development server...")
    print("Access the application at: http://localhost:5000")
    print("Debug mode is enabled - errors will show detailed tracebacks")
    print("Press Ctrl+C to stop the server")
    
    app.run(
        host='0.0.0.0',
        port=5000,
        debug=True,
        use_reloader=True
    )
"""

print("To run this Flask application outside of Jupyter:")
print("1. Save the code as a .py file")
print("2. Run: python your_file.py")
print("3. Access http://localhost:5000 in your browser")

### Starting Flask Development Server

To run this Flask application outside of Jupyter:
1. Save the code as a .py file
2. Run: python your_file.py
3. Access http://localhost:5000 in your browser


### 8.2 Production Deployment Notes

display(Markdown("### Production Deployment Checklist"))

checklist = """
- [ ] Set `DEBUG = False`
- [ ] Set `ENV = 'production'`
- [ ] Use environment variables for secrets
- [ ] Configure proper logging
- [ ] Set up monitoring/alerting
- [ ] Implement rate limiting
- [ ] Use production WSGI server (Gunicorn, uWSGI)
- [ ] Configure reverse proxy (Nginx, Apache)
- [ ] Set up SSL/TLS certificates
- [ ] Implement backup strategies
"""

display(Markdown(checklist))

### 9. Summary and Best Practices

display(Markdown("### Error Handling Best Practices"))

best_practices = """
1. **Always use custom error pages** - Never show raw errors to users in production
2. **Implement proper logging** - Log errors with context for debugging
3. **Use appropriate HTTP status codes** - Help clients understand what went wrong
4. **Handle exceptions gracefully** - Don't let uncaught exceptions crash your app
5. **Provide meaningful error messages** - But don't expose sensitive information
6. **Test error scenarios** - Ensure your error handling works as expected
7. **Monitor and alert** - Set up monitoring for critical errors
8. **Use structured logging** - Makes log analysis easier
9. **Implement circuit breakers** - For external service dependencies
10. **Document error responses** - Especially for API endpoints
"""

display(Markdown(best_practices))

display(Markdown("### Debugging Tips"))

debugging_tips = """
1. **Use Flask's debug mode** during development
2. **Leverage the interactive debugger** for troubleshooting
3. **Add strategic logging** at key points in your code
4. **Use Flask-DebugToolbar** for detailed request information
5. **Test with different data inputs** to find edge cases
6. **Reproduce errors consistently** before attempting fixes
7. **Check application logs** for additional context
8. **Use version control** to track changes when debugging
9. **Isolate the problem** by creating minimal test cases
10. **Ask for help** with specific, reproducible examples
"""

display(Markdown(debugging_tips))

#### 10. Quick Reference

display(Markdown("### Quick Reference Code Snippets"))

snippets = """
```python
# Basic error handler
@app.errorhandler(404)
def not_found(error):
    return render_template('404.html'), 404

# JSON error response
@app.errorhandler(500)
def internal_error(error):
    return jsonify({'error': 'Internal server error'}), 500

# Custom exception
class CustomError(Exception):
    def __init__(self, message, status_code=400):
        self.message = message
        self.status_code = status_code

# Logging setup
import logging
logging.basicConfig(level=logging.INFO)

# Debug mode
app.run(debug=True)  # Development only!


```python
# Final summary
display(Markdown("## 🎉 Flask Error Handling Notebook Complete!"))

summary = """
This notebook has covered:

✅ **Common error types** in Flask applications  
✅ **Debug configuration** for development vs production  
✅ **Custom error handlers** for different scenarios  
✅ **Advanced debugging techniques** and tools  
✅ **Error recovery patterns** and fallback strategies  
✅ **Best practices** for robust error handling  

**Next Steps:**
1. Implement these patterns in your Flask projects
2. Customize error pages for your application
3. Set up proper logging and monitoring
4. Test your error handling thoroughly

Remember: Good error handling makes your application more reliable and user-friendly!
"""

display(Markdown(summary))